<a href="https://colab.research.google.com/github/arashash/Pawpularity/blob/main/duplicate_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from psutil import *
cpu_count()

2


**Install packages:**

In [2]:
# utility packages for pytorch
!pip install -q imagehash
!pip install -q pandarallel

     |████████████████████████████████| 812 kB 9.5 MB/s 


**Download datasets and scripts:**

In [3]:
# dowload the pawpularity dataset 
!gdown --id 1bqwF937JkUp08cuBGl3joERkfSYWCUCg
!mkdir ../content/petfinder-pawpularity-score
!unzip --qq petfinder-pawpularity-score.zip -d ../content/petfinder-pawpularity-score
!rm petfinder-pawpularity-score.zip

# dowload the train+test of the adoption dataset 
!gdown --id 13gfFCHPhmYzgQ1y_piV9hJuWnEW9qM7V
!mkdir ../content/all_adoption_images
!unzip --qq all_adoption_images.zip -d ../content/all_adoption_images
!rm all_adoption_images.zip

# dowload the splitter script
!gdown --id 1pm2QMUcH9tSnCUB-XbCtEgvFxjf62sqF

Downloading...
From: https://drive.google.com/uc?id=1bqwF937JkUp08cuBGl3joERkfSYWCUCg
To: /content/petfinder-pawpularity-score.zip
100% 1.03G/1.03G [00:08<00:00, 124MB/s]
Downloading...
From: https://drive.google.com/uc?id=13gfFCHPhmYzgQ1y_piV9hJuWnEW9qM7V
To: /content/all_adoption_images.zip
100% 1.98G/1.98G [00:44<00:00, 44.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1pm2QMUcH9tSnCUB-XbCtEgvFxjf62sqF
To: /content/folder_splitter.py
100% 1.33k/1.33k [00:00<00:00, 601kB/s]


In [23]:
import os
import cv2
import torch
import imagehash
import pickle
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
import shutil
import csv 
from csv import writer
import matplotlib.pylab as plt
import math
from tqdm import tqdm
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from pandarallel.utils import progress_bars
progress_bars.is_notebook_lab = lambda : True

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
# number of images initially
pawpularity_imgs = next(os.walk('/content/petfinder-pawpularity-score/train'))[2]
adoption_imgs = next(os.walk('/content/all_adoption_images'))[2]
print('Number of pawpularity: %d, and adoption images: %d' %(len(pawpularity_imgs), len(adoption_imgs)))

Number of pawpularity: 9912, and adoption images: 72776


In [6]:
# Divide adoption folder into sub-folders
num_subfolders = 5
N = math.ceil(len(adoption_imgs)/num_subfolders) # number of images in each sub-folder
!python folder_splitter.py $'/content/all_adoption_images' $N # run the splitter script

In [30]:
# number of images in each sub-folder
sub_imgs = [0]*num_subfolders
for i in range(num_subfolders):
  sub_imgs[i] = next(os.walk('/content/all_adoption_images/'+str((i+1)).zfill(3)))[2]

print('Initial number of images:')
print('Adoption images in eash sub-folder: %d, %d, %d, %d and %d' %(len(sub_imgs[0]), len(sub_imgs[1]), len(sub_imgs[2]), len(sub_imgs[3]), len(sub_imgs[4])))
print('Total adoption images: %d', sum(len(sub_imgs)))

pawpularity_imgs = next(os.walk('/content/petfinder-pawpularity-score/train'))[2]
print('Pawpularity images: %d' %len(pawpularity_imgs))

Initial number of images:
Adoption images in eash sub-folder: 14556, 14556, 14556, 14556 and 14552
Pawpularity images: 9912


In [19]:
def create_csv(image_path, csv_path, csv_name):
  if not os.path.exists(csv_path):
    os.makedirs(csv_path)
  names = []
  content = os.listdir(image_path)
  for image in content: 
        names.append(image)     
  new_names= [[x] for x in names]
  with open(csv_path+csv_name+'.csv', 'w', newline='') as csvfile: 
      writer=csv.writer(csvfile)
      writer.writerow(['Name'])
      writer.writerows(new_names)   

In [9]:
# Compute hash values for each image
def load_image_and_hash(series):
    funcs = [
        imagehash.average_hash,
        imagehash.phash,
        imagehash.dhash,
        imagehash.whash,
    ]

    image = Image.open(series.file)
    return np.array([f(image).hash for f in funcs]).reshape(256).astype(np.uint8)

In [10]:
def count_similar_images(image_path, csv_path): 
  # Add two columns to the csv file of image's names: first the path to that image and second it's hash value
  train_df = ( 
      pd.read_csv(csv_path)
      .assign(file=lambda x: image_path + '/' + x.Name)
      .assign(hashes=lambda x: x.parallel_apply(load_image_and_hash, axis=1))
  )

  print('\nStart comparing the hashes...\n')
  train_hashes = torch.from_numpy(np.array(train_df.hashes.to_list()))
  sims = np.array([(train_hashes[i] == train_hashes).sum(dim=1).cpu().numpy() for i in range(train_hashes.shape[0])]) / 256

  indices1 = np.where(sims > 0.9)
  indices2 = np.where(indices1[0] != indices1[1])
  petids1 = train_df.iloc[indices1[0][indices2]].Name.values
  petids2 = train_df.iloc[indices1[1][indices2]].Name.values
  duplicates = set(list(petids1))

  return duplicates

In [11]:
def copy_unique_images(duplicates, source_path, dest_path):
  if not os.path.exists(dest_path): os.makedirs(dest_path)
  imgs = next(os.walk(source_path))[2]
  for i, this_img in enumerate(imgs):
    if this_img not in duplicates: 
      shutil.copy(os.path.join(source_path, this_img), dest_path)

In [59]:
# copy unique images for pawpularity dataset
create_csv(pawpularity_path, '/content/csv_files/', 'pawpularity')
duplicates = count_similar_images(image_path=pawpularity_path, csv_path='/content/csv_files/pawpularity.csv')
copy_unique_images(duplicates,pawpularity_path,'/content/unique_pawpularity_images_1/')

initial_images_count = os.listdir(pawpularity_path)
copied_images_count = os.listdir('/content/unique_pawpularity_images_1/')
print('Pawpularity images:')
print('Initial images (with duplicates):%d, num of duplicates: %d, and copied images after duplicate removal: %d' 
      %(len(initial_images_count), len(duplicates), len(copied_images_count)))
print('*** Done! ***\n')


Start comparing the hashes...

Pawpularity images:
Initial images (with duplicates):9912, num of duplicates: 54, and copied images after duplicate removal: 9858
*** Done! ***



In [60]:
# copy unique images for adoption dataset
for i in range(num_subfolders):
  create_csv(adoption_path[i], '/content/csv_files/', 'adoption_'+str(i+1).zfill(3)) 
  duplicates = count_similar_images(image_path=adoption_path[i], csv_path='/content/csv_files/adoption_'+str(i+1).zfill(3)+'.csv')
  copy_unique_images(duplicates, source_path=adoption_path[i], dest_path='/content/unique_adoption_images_1/'+str(i+1).zfill(3)+'/')
  
  initial_images_count = os.listdir(adoption_path[i])
  copied_images_count = os.listdir('/content/unique_adoption_images_1/'+str(i+1).zfill(3))
  
  print('Initial images (with duplicates):%d, num of duplicates: %d, and copied images after duplicate removal: %d' 
        %(len(initial_images_count), len(duplicates), len(copied_images_count)))
  print('*** Done! ***\n')


Start comparing the hashes...

Initial images (with duplicates):14556, num of duplicates: 259, and copied images after duplicate removal: 14297
*** Done! ***




Start comparing the hashes...

Initial images (with duplicates):14556, num of duplicates: 216, and copied images after duplicate removal: 14340
*** Done! ***




Start comparing the hashes...

Initial images (with duplicates):14556, num of duplicates: 234, and copied images after duplicate removal: 14322
*** Done! ***




Start comparing the hashes...

Initial images (with duplicates):14556, num of duplicates: 237, and copied images after duplicate removal: 14319
*** Done! ***




Start comparing the hashes...

Initial images (with duplicates):14552, num of duplicates: 193, and copied images after duplicate removal: 14359
*** Done! ***



In [72]:
# Path to pawpularity and adoption folders with unique images within each folder
unique_pawpularity_path ='/content/unique_pawpularity_images_1'
unique_adoption_path = list(os.path.join('/content/unique_adoption_images_1/', str((i+1)).zfill(3)) for i in range(num_subfolders))

In [73]:
merged_source_path = [[unique_pawpularity_path, unique_adoption_path[i]] for i in range(num_subfolders)]
merged_dest_path = list(os.path.join('/content/merged_1/images/', str((i+1)).zfill(3)) for i in range(num_subfolders))
for dir in range(len(merged_dest_path)):
  if not os.path.exists(merged_dest_path[dir]):
    os.makedirs(merged_dest_path[dir])

In [75]:
# For each part of the previous comp, merge it with the current comp into a new merged folder
for i in range(len(merged_source_path)): 
  this_dir = merged_source_path[i] # folders to be merged
  content_list = {}
  for index, val in enumerate(this_dir): # enumerate on this_dir to get the content of all the folders and store it in a dictionary
    path = os.path.join(val)
    content_list[ this_dir[index] ] = os.listdir(path)   

  for sub_dir in content_list: # loop through the list of folders   
      for contents in content_list[sub_dir]: # loop through the contents of the list of folders       
          path_to_content = sub_dir + "/" + contents # make the path of the content to move        
          shutil.copy(path_to_content, merged_dest_path[i]) # move the file to the merged folder

In [77]:
# Calculate number of images after merge 
print('Number of images after merging (yet before removing between comp duplicates):')
for i in range(len(merged_dest_path)):
  imgs = next(os.walk(merged_dest_path[i]))[2]
  print(len(imgs))

Number of images after merging:
24155
24198
24180
24177
24217


In [78]:
# copy unique images for merged dataset
for i in range(num_subfolders):
  create_csv(merged_dest_path[i], '/content/csv_files/', 'merged_'+str(i+1).zfill(3))
  duplicates = count_similar_images(image_path=merged_dest_path[i], csv_path='/content/csv_files/merged_'+str(i+1).zfill(3)+'.csv')
  copy_unique_images(duplicates, 
                     source_path=merged_dest_path[i], 
                     dest_path='/content/unique_merged_images_1/'+str(i+1).zfill(3)+'/')
  
  initial_images_count = os.listdir(merged_dest_path[i])
  copied_images_count = os.listdir('/content/unique_merged_images_1/'+str(i+1).zfill(3))
  
  print('Initial images (with duplicates):%d, num of duplicates: %d, and copied images after duplicate removal: %d' 
        %(len(initial_images_count), len(duplicates), len(copied_images_count)))
  print('*** Done! ***\n')


Start comparing the hashes...

Initial images (with duplicates):24155, num of duplicates: 786, and copied images after duplicate removal: 23369
*** Done! ***




Start comparing the hashes...

Initial images (with duplicates):24198, num of duplicates: 848, and copied images after duplicate removal: 23350
*** Done! ***




Start comparing the hashes...

Initial images (with duplicates):24180, num of duplicates: 890, and copied images after duplicate removal: 23290
*** Done! ***




Start comparing the hashes...

Initial images (with duplicates):24177, num of duplicates: 852, and copied images after duplicate removal: 23325
*** Done! ***




Start comparing the hashes...

Initial images (with duplicates):24217, num of duplicates: 882, and copied images after duplicate removal: 23335
*** Done! ***



In [80]:
# Calculate number of final images 
print('Number of images in merged folders after removing between comp duplicates:')
for i in range(num_subfolders):
  imgs = next(os.walk('/content/unique_merged_images_1/'+str(i+1).zfill(3)+'/'))[2]
  print(len(imgs))

Number of images in merged folders after removing between comp duplicates:
23369
23350
23290
23325
23335


In [92]:
# merge five folds of the final folder
pawpularity_imgs = next(os.walk('/content/petfinder-pawpularity-score/train'))[2]
os.makedirs('./new_unlabeled_images/')
count = [0]*num_subfolders
for i in range(num_subfolders): 
  imgs = next(os.walk('/content/unique_merged_images_1/'+str(i+1).zfill(3)+'/'))[2]
  for idx, this_img in enumerate(imgs):
    if this_img not in pawpularity_imgs:
      count[i] += 1
      shutil.copy(os.path.join('/content/unique_merged_images_1/'+str(i+1).zfill(3), this_img), './new_unlabeled_images/')

new_imgs = next(os.walk('./new_unlabeled_images/'))[2]
print('Total final images: %d' %len(new_imgs))
print('Unique images in each adoption fold (after removal of duplicates): %d, %d, %d, %d, %d' %(count[0], count[1], count[2], count[3], count[4]))

Total final images: 69508
Unique images in each adoption fold (after removal of duplicates): 13904, 13916, 13877, 13893, 13918


In [93]:
shutil.make_archive('new_unlabeled_images/', 'zip', '/content/new_unlabeled_images/')

'/content/new_unlabeled_images.zip'

In [95]:
create_csv('/content/new_unlabeled_images', '/content/csv_files/', 'new_unlabeled_images')

In [ ]:
# from matplotlib_venn import venn2 
# from matplotlib import pyplot as plt
  
# # depict venn diagram
# venn2(subsets = (sum(len_folders[1:6])-sum(sim_images_count), len_folders[0]-sum(sim_images_count), sum(sim_images_count)), 
#       set_labels = ('Previous comp:%d'%sum(len_folders[1:6]), 'Current comp:%d'%len_folders[0]))
# plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# img = mpimg.imread('/content/all_adoption_images/001/5bd725e47-3.jpg')
# imgplot = plt.imshow(img)
# plt.show()